<a href="https://colab.research.google.com/github/mohigot/Customer-Segmentation/blob/main/Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
from google.colab import drive 

drive.mount('/content/drive')

path = "/content/drive/MyDrive/task1.xlsx"
data = pd.read_excel(path)

In [ ]:
import numpy as np
import scipy.stats
import pandas as pd 
import numpy as np
import scipy.stats
from google.colab import drive 
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.neighbors import DistanceMetric
import plotly 
import plotly.express as px
from scipy.stats import chi2_contingency


print(data.dtypes)
data.describe().T

invoiceid                int64
userid                   int64
productid                int64
date            datetime64[ns]
amount(Rial)             int64
dtype: object


,count,mean,std,min,25%,50%,75%,max
invoiceid,306.0,3.730525e+05,4.635225e+04,303365.0,333429.25,367623.0,417218.50,462582.0
userid,306.0,8.441242e+02,6.436839e+02,8.0,212.75,783.0,1314.00,2040.0
productid,306.0,4.905980e+02,1.965603e+02,5.0,436.00,483.0,663.75,887.0
amount(Rial),306.0,3.243603e+06,4.907738e+06,27533.0,690000.00,1790000.0,3568250.00,52740000.0


In [ ]:
print("median:","\n",data.median(numeric_only=True),"\n")

In [ ]:
import missingno as msno
msno.matrix(data) #checking completeness

In [ ]:
for col in data:
  sns.histplot(data[col] , color="lightblue", kde=True, linewidth=0)
  plt.show()  

In [ ]:
data.hist(column='amount(Rial)');

In [ ]:
def detect_outlier(dt):
    # find q1 and q3 values
    q1, q3 = np.percentile(sorted(dt), [25, 75])

    # compute IRQ
    iqr = q3 - q1

    # find lower and upper bounds
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)

    outliers = [x for x in dt if x <= lower_bound or x >= upper_bound]

    return outliers

ax = sns.boxplot(x=data['amount(Rial)'])
print(detect_outlier(data['amount(Rial)']))

In [ ]:
datalist = data.groupby(["userid"], as_index=False).sum().sort_values("amount(Rial)", ascending=False)
datalist

In [ ]:
temp = data.groupby(by=['userid', 'invoiceid'], as_index=False)['date'].count()
nb_products_per_basket = temp.rename(columns = {'date':'Number of products'})
nb_products_per_basket.sort_values('userid')
temp = nb_products_per_basket.groupby('userid', as_index=False).sum()

temp

In [ ]:
price_range = [0, 1000000, 2000000, 3000000, 4000000, 5000000, 6000000]
count_price = []
for i, price in enumerate(price_range):
    if i == 0: continue
    val = datalist[(datalist['amount(Rial)'] < price) &
                       (datalist['amount(Rial)'] > price_range[i-1])]['amount(Rial)'].count()
    count_price.append(val)

   
plt.rc('font', weight='bold')
f, ax = plt.subplots(figsize=(11, 6))
colors = ['yellowgreen', 'gold', 'wheat', 'c', 'violet', 'royalblue','firebrick']
labels = [ '{}-{}'.format(price_range[i-1], s) for i,s in enumerate(price_range) if i != 0]
sizes  = count_price
explode = [0.0 if sizes[i] < 100 else 0.0 for i in range(len(sizes))]
ax.pie(sizes, explode = explode, labels=labels, colors = colors,
       autopct = lambda x:'{:1.0f}%'.format(x) if x > 1 else '',
       shadow = False, startangle=0)
ax.axis('equal')
f.text(0.5, 1.01, "Purchase based on Basket Price", ha='center', fontsize = 18);

In [ ]:
purchaseDate = data.groupby(['userid'], as_index=False)
purchaseDate = purchaseDate.agg(FirstPurachase= ('date', np.min), LastPurchase =('date', np.max))
purchaseDate

In [ ]:
transactions_per_user = data.groupby(['userid'], as_index=False)
transactions_per_user = transactions_per_user['amount(Rial)'].agg(['count','min','max','mean','sum'])
transactions_per_user

In [ ]:
info = pd.merge(transactions_per_user, purchaseDate, how='inner', on = 'userid')
info = info.rename(columns = {'count':'Frequency', 'sum':'Monetary',
                              'min': 'minPurchase', 'max':'maxPurchase', 'mean':'meanPurchase'})
info

In [ ]:
data_log = data[['amount(Rial)']].apply(np.log, axis=1).round(3)
sns.distplot(data_log['amount(Rial)'])

#Customer Segmentation

RFM

In [ ]:
PurchaseInfo = info.drop(["CustomerGroup"],axis = 1)


last_date = data['date'].max().date()

last_purchase  = pd.DataFrame(data.groupby(by=['userid'])['date'].max())

test2 = last_purchase.applymap(lambda x:(last_date - x.date()).days)

PurchaseInfo.loc[:, 'Recency'] = test2.reset_index(drop = False)['date']

PurchaseInfo

In [ ]:
rfm = PurchaseInfo[['userid','Recency', 'Frequency', 'Monetary']]

In [ ]:
quantiles = rfm.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()
quantiles

In [ ]:
#Functions to create R, F and M segments

def RScoring(x,p,d):
    if x <= d[p][0.50]:
        return 1
    else:
        return 0

def FnMScoring(x,p,d):
  if x <= d[p][0.5]:
      return 0
  else:
      return 1

In [ ]:
#Calculate Add R, F and M segment value columns in the existing dataset to show R, F and M segment values

rfm['R'] = rfm['Recency'].apply(RScoring, args=('Recency',quantiles,))
rfm['F'] = rfm['Frequency'].apply(FnMScoring, args=('Frequency',quantiles,))
rfm['M'] = rfm['Monetary'].apply(FnMScoring, args=('Monetary',quantiles,))
rfm

In [ ]:
rfm['RFMGroup'] = rfm.R.map(str) + rfm.F.map(str) + rfm.M.map(str)

def binaryToInt(gp_str):
    return int(gp_str, 2)

rfm['RFMScore'] = rfm['RFMGroup'].apply(binaryToInt)
rfm

In [ ]:
#Assign Loyalty Level to each customer
Loyalty_Level = {0 :'Uncertain', 1 : 'Spender', 2 : 'Frequent', 3 : 'Churn', 4 : 'First Time', 5 : 'Valuable', 6 : 'Shopper', 7 : 'Best'}
rfm["RFM_Loyalty_Level"] = rfm["RFMScore"].map(Loyalty_Level)

In [ ]:
rfm.to_excel(r'/content/drive/MyDrive/CustomerSegmentation.xlsx', index = False)

*Segmentation by mean*

In [ ]:
PurchaseInfo['Frequency'].mean()
PurchaseInfo['Recency'].mean()
PurchaseInfo['Monetary'].mean()

In [ ]:
r = 150
f = 2
m = 8004376

def flag_df(df): 
    if (df['Frequency'] >= f) and (df['Monetary'] >= m) and (df['Recency'] < r):
        return 'Best'
    elif (df['Frequency'] < f) and (df['Monetary'] >= m) and (df['Recency'] < r):
        return 'Valuable'
    elif (df['Frequency'] >= f) and (df['Monetary'] < m) and (df['Recency'] < r):
        return 'Shopper'
    elif (df['Frequency'] < f) and (df['Monetary'] < m) and (df['Recency'] < r):
        return 'First Time'
    elif (df['Frequency'] >= f) and (df['Monetary'] >= m) and (df['Recency'] >= r):
        return 'Churn'
    elif (df['Frequency'] >= f) and (df['Monetary'] < m) and (df['Recency'] >= r):
        return 'Frequent'
    elif (df['Frequency'] < f) and (df['Monetary'] >= m) and (df['Recency'] >= r):
        return 'Spender'
    elif (df['Frequency'] < f) and (df['Monetary'] < m) and (df['Recency'] >= r):
        return 'Uncertain'
    
PurchaseInfo['CustomerType'] = PurchaseInfo.apply(flag_df, axis = 1)

In [ ]:
rfm.to_excel(r'/content/drive/MyDrive/CustomerSegmentationByMean.xlsx', index = False)

RFM Skewness

In [ ]:
plt.figure(figsize=(20,5))

plt.subplot(1,3,1)
sns.distplot(rfm['Recency'])

plt.subplot(1,3,2)
sns.distplot(rfm['Frequency'])

plt.subplot(1,3,3)
sns.distplot(rfm['Monetary'])

kmeans

In [ ]:
#normalize data 

import pandas as pd
from sklearn import preprocessing

x = rfm[['Recency','Frequency', 'Monetary']].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
rfm_scaled = pd.DataFrame(x_scaled)

In [ ]:
#wcss stands for within cluster sum of squares
wcss={}

for k in range(1,15):
    km = KMeans(n_clusters=k, init='k-means++', max_iter=1000 , random_state=20)
    km = km.fit(rfm_scaled)
    wcss[k] = km.inertia_
    
#Plot graph for the wcss values and the number of clusters
sns.pointplot(x=list(wcss.keys()), y=list(wcss.values()))
plt.xlabel('Number of clusters K')
plt.ylabel('Sum of square distances')
plt.title('Elbow method for optimal K')
plt.show()

In [ ]:
from sklearn.cluster import KMeans

plt.figure(figsize=(7,7))

##Scatter Plot Frequency Vs Recency
Colors = ["red", "green", "blue", "yellow", "brown"]
rfm['Color'] = rfm['Cluster'].map(lambda p: Colors[p])
ax = rfm.plot(    
    kind="scatter", 
    x="Recency", y="Frequency",
    figsize=(10,8),
    c = rfm['Color'])

#Customer Behavior

Daily Sales by Day of the Week

In [ ]:
data_cp = data.copy()

data_cp['weekday'] = data_cp['date'].dt.day_name()

plt.figure(figsize=(8, 6))
plt.title('Daily Sales by Day of the Week')
sns.countplot(data_cp['weekday'])

In [ ]:
plt.figure(figsize=(8, 6))
plt.title('monthly Sales')
sns.countplot(data_cp['month'])

In [ ]:
plt.figure(figsize=(8, 6))
plt.title('RFM_Loyalty_Level count')
sns.countplot(rfm['RFM_Loyalty_Level'])

In [ ]:
dtl = data.groupby(["productid"], as_index=False).sum().sort_values("productid", ascending=False)
dtl

In [ ]:
x = data_cp.groupby('productid')['amount(Rial)'].sum()[:20].sort_values()
x.plot(kind='barh')
plt.xlabel('amount(Rial)')

*apriori*

In [ ]:
!pip install apyori
!pip install efficient-apriori

from apyori import apriori
#from mlxtend.frequent_patterns import apriori, association_rules

data_selection = data[['productid']]

records = []
for i in range(0, 306):
    records.append([str(data_selection.values[i,j]) for j in range(0, 1)])

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(records).transform(records)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

In [ ]:
dt_user = data[['userid']].copy() 
fr_items = pd.merge(dt_user, df, left_index=True, right_index=True)
fr_items

Frequent Itemsests

In [ ]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df, min_support = 0.00005, use_colnames=True)

frequent_itemsets